### VRP

In [11]:
import torch
import networkx as nx
import os
import numpy as np
from torch_geometric.data import Data, DataLoader
from VRP.creat_vrp import reward1,creat_instance
from VRP.VRP_Actor import Model

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def rollout(model, dataset,n_nodes):
    model.eval()
    def eval_model_bat(bat):
        with torch.no_grad():
            cost, _ = model(bat, n_nodes * 2,True)
            cost = reward1(bat.x,cost.detach(), n_nodes)
        return cost.cpu()
    totall_cost = torch.cat([eval_model_bat(bat.to(device))for bat in dataset], 0)
    return totall_cost

def evaliuate(valid_loder,n_node):
    folder = 'trained'

    agent = Model(3, 128, 1, 16, conv_laysers=4).to(device)
    agent.to(device)

    filepath = os.path.join(folder, '%s' % n_node)

    if os.path.exists(filepath):
        path1 = os.path.join(filepath, 'actor.pt')
        agent.load_state_dict(torch.load(path1, device))
    cost = rollout(agent, valid_loder, n_node)
    cost = cost.mean()
    print('Problem:VRP''%s' % n_node,'/ Average distance:',cost.item())

    cost1=cost.min()
    return cost,cost1



In [13]:

datas = []
n_node = 21
if n_node==21 or n_node==51 or n_node==101:
    node_ = np.loadtxt('VRP/test_data/vrp{}_test_data.csv'.format(n_node-1), dtype=float, delimiter=',')
    demand_ = np.loadtxt('VRP/test_data/vrp{}_demand.csv'.format(n_node-1), dtype=float, delimiter=',')
    capcity_ = np.loadtxt('VRP/test_data/vrp{}_capcity.csv'.format(n_node-1), dtype=float, delimiter=',')
    batch_size=128
else:
    print('Please enter 21, 51 or 101')
node_ = node_.reshape(-1, n_node, 2)
# Calculate the distance matrix
def c_dist(x1,x2):
    return ((x1[0]-x2[0])**2+(x1[1]-x2[1])**2)**0.5
#edges = torch.zeros(n_nodes,n_nodes)

data_size = node_.shape[0]

edges = np.zeros((data_size, n_node, n_node, 1))
for k, data in enumerate(node_):
    for i, (x1, y1) in enumerate(data):
        for j, (x2, y2) in enumerate(data):
            d = c_dist((x1, y1), (x2, y2))
            edges[k][i][j][0] = d
edges_ = edges.reshape(data_size, -1, 1)

edges_index = []
for i in range(n_node):
    for j in range(n_node):
        edges_index.append([i, j])
edges_index = torch.LongTensor(edges_index)
edges_index = edges_index.transpose(dim0=0, dim1=1)

for i in range(2): #номер датафрейма(вроде решается для всех сразу)
    data = Data(x=torch.from_numpy(node_[i]).float(), edge_index=edges_index, edge_attr=torch.from_numpy(edges_[i]).float(),
                demand=torch.tensor(demand_[i]).unsqueeze(-1).float(),
                capcity=torch.tensor(capcity_[i]).unsqueeze(-1).float())
    datas.append(data)
dl = DataLoader(datas, batch_size=1)


In [14]:

evaliuate(dl,n_node)

Problem:VRP21 / Average distance: 13.29855728149414


(tensor(13.2986), tensor(13.2986))

### matplotlib

In [4]:
import os
import numpy as np
import torch
from VRP.creat_vrp import reward1

from torch_geometric.data import Data,DataLoader
from VRP.VRP_Actor import Model
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def discrete_cmap(N, base_cmap=None):
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [6]:
def plot_vehicle_routes(data, route, ax1,Greedy, markersize=5, visualize_demands=False, demand_scale=1, round_demand=False):

    plt.rc('font', family='Times New Roman', size=10)

    routes = [r[r != 0] for r in np.split(route.cpu().numpy(), np.where(route.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)
    depot = data.x[0].cpu().numpy()
    locs = data.x[1:].cpu().numpy()
    demands = data.demand.cpu().numpy()*10
    demands=demands[1:]

    capacity = data.capcity*10

    x_dep, y_dep = depot
    ax1.plot(x_dep, y_dep, 'sk', markersize=markersize * 4)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)

    legend = ax1.legend(loc='upper center')

    cmap = discrete_cmap(len(routes) + 2, 'nipy_spectral')
    dem_rects = []
    used_rects = []
    cap_rects = []
    qvs = []
    total_dist = 0
    for veh_number, r in enumerate(routes):
        color = cmap(len(routes) - veh_number)  # Invert to have in rainbow order

        route_demands = demands[r - 1]
        coords = locs[r - 1, :]
        xs, ys = coords.transpose()

        total_route_demand = sum(route_demands)
        #assert total_route_demand <= capacity
        if not visualize_demands:
            ax1.plot(xs, ys, 'o', mfc=color, markersize=markersize, markeredgewidth=0.0)

        dist = 0
        x_prev, y_prev = x_dep, y_dep
        cum_demand = 0
        for (x, y), d in zip(coords, route_demands):
            dist += np.sqrt((x - x_prev) ** 2 + (y - y_prev) ** 2)
            cap_rects.append(Rectangle((x, y), 0.01, 0.1))
            used_rects.append(Rectangle((x, y), 0.01, 0.1 * total_route_demand / capacity))
            dem_rects.append(Rectangle((x, y + 0.1 * cum_demand / capacity), 0.01, 0.1 * d / capacity))

            x_prev, y_prev = x, y
            cum_demand += d

        dist += np.sqrt((x_dep - x_prev) ** 2 + (y_dep - y_prev) ** 2)
        total_dist += dist
        qv = ax1.quiver(
            xs[:-1],
            ys[:-1],
            xs[1:] - xs[:-1],
            ys[1:] - ys[:-1],
            scale_units='xy',
            angles='xy',
            scale=1,
            color=color,
            label='R{}, N({}), C {} / {}, D {:.2f}'.format(
                veh_number,
                len(r),
                int(total_route_demand) if round_demand else total_route_demand,
                int(capacity) if round_demand else capacity,
                dist
            )
        )

        qvs.append(qv)
    if Greedy:
        ax1.set_title('Greedy,{} routes, total distance {:.2f}'.format(len(routes), total_dist), family='Times New Roman',size=20)
    else:
        ax1.set_title('Sampling1280,{} routes, total distance {:.2f}'.format(len(routes), total_dist), family='Times New Roman',size=20)

    ax1.legend(handles=qvs)
    plt.legend(loc=1)
    pc_cap = PatchCollection(cap_rects, facecolor='whitesmoke', alpha=1.0, edgecolor='lightgray')
    pc_used = PatchCollection(used_rects, facecolor='lightgray', alpha=1.0, edgecolor='lightgray')
    pc_dem = PatchCollection(dem_rects, facecolor='black', alpha=1.0, edgecolor='black')

    if visualize_demands:
        ax1.add_collection(pc_cap)
        ax1.add_collection(pc_used)
        ax1.add_collection(pc_dem)
    plt.show()
    #plt.savefig("./temp{}.png".format(54), dpi=600, bbox_inches='tight')


In [262]:
def get_orders_by_date(date, daily_orders, GNN):
    if GNN == True:
        try:
            orders = daily_orders.loc[date]
            demand = np.array(orders)
            # demand[demand== 0] = 500
            demand[0] = 0
            demand = demand/1000
            float_array = demand.astype(float)
            float_array = np.round(float_array, 3)
            return float_array
        except KeyError:
            return np.array([])  # Если дата не найдена, возвращаем пустой список
    else:
        try:
            orders = daily_orders.loc[date]
            orders[0] = 0
            return orders.tolist()
        except KeyError:
            return []  # Если дата не найдена, возвращаем пустой список

In [229]:
import pandas as pd
example_data_path = "./VRP/exp_data/"
demands = pd.read_csv(os.path.join(example_data_path, "second_scenario.csv"))
demands.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
# Предположим, что ваш DataFrame называется orders_df
df = demands
# Преобразуем столбец 'date' в datetime
df['date'] = pd.to_datetime(df['date'])

# Устанавливаем 'date' в качестве индекса
df.set_index('date', inplace=True)

# Агрегируем данные по дням и суммируем заказы
daily_orders = df.resample('D').sum()


input_date = '2023-08-20'  # Замените на нужную дату
demand = get_orders_by_date(input_date, daily_orders, GNN = True)
demand


array([0.   , 0.458, 0.462, 0.508, 0.705, 0.27 , 0.677, 0.5  , 0.545,
       0.637, 0.658, 0.703, 0.548, 0.466, 0.408, 0.58 , 0.669, 0.71 ,
       0.521, 0.881, 0.781, 0.749, 0.808, 0.815, 0.963, 1.001, 0.836,
       0.54 , 0.847, 1.035, 0.635, 0.917, 0.865, 0.62 , 0.724, 0.663,
       0.954, 0.643, 0.339, 0.565])

In [157]:

demand_=np.loadtxt('VRP/test_data/vrp50_demand.csv', dtype=float, delimiter=',')
demand_= demand_[:1]
demand_ = demand_[0][:40]
demand_

array([0. , 0.2, 0.8, 0.2, 0.5, 0.9, 0.9, 0.4, 0.7, 0.1, 0.1, 0.8, 0.3,
       0.9, 0.8, 0.3, 0.8, 0.4, 0.9, 0.3, 0.6, 0.2, 0.5, 0.6, 0.6, 0.9,
       0.2, 0.1, 0.3, 0.3, 0.4, 0.4, 0.7, 0.2, 0.8, 0.3, 0.6, 0.8, 0.2,
       0.6])

In [263]:
Greedy=False
n_nodes = 40
actor = 21

# node_ = np.loadtxt('VRP/test_data/vrp50_test_data.csv', dtype=float, delimiter=',')
# # demand_=np.loadtxt('VRP/test_data/vrp50_demand.csv', dtype=float, delimiter=',')
# capcity_=np.loadtxt('VRP/test_data/vrp50_capcity.csv', dtype=float, delimiter=',')

# demand = get_orders_by_date(input_date, daily_orders, array = True)
# demand_ = demand[0][:n_nodes]
# demand_[demand_ == 0] = 500
# demand_=demand_/1000
# demand_ = np.round(demand_, 1)

# node_ = node_[:n_nodes]

# capcity_ = np.array([5.]) #настройка вместимости транспорта

node_ = np.loadtxt('VRP/test_data/vrp50_test_data.csv', dtype=float, delimiter=',')
node_ = node_[:n_nodes]

# demand_=np.loadtxt('VRP/test_data/vrp50_demand.csv', dtype=float, delimiter=',')
# demand_= demand_[:1]
# demand_ = demand_[0][:n_nodes]
input_date = '2023-08-20' 
demand_ = get_orders_by_date(input_date, daily_orders, GNN = True)

capcity_ = np.array([8.]) #настройка вместимости транспорта

node_,demand_=node_.reshape(-1,n_nodes,2),demand_.reshape(-1,n_nodes)

# data_size = node_.shape[0]
# print(data_size)
# x=np.random.randint(1,data_size)

x=0 # выбираем набор данных

# Calculate the distance matrix
edges = np.zeros((n_nodes, n_nodes, 1))
def c_dist(x1, x2):
    # return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** 0.5
    return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** 0.5
for i, (x1, y1) in enumerate(node_[x]):
    for j, (x2, y2) in enumerate(node_[x]):
        d = c_dist((x1, y1), (x2, y2))
        edges[i][j][0] = d
edges_ = edges.reshape(-1, 1)
# edges_ = np.round(edges_).astype(int)

edges_index = []
for i in range(n_nodes):
    for j in range(n_nodes):
        edges_index.append([i, j])
edges_index = torch.LongTensor(edges_index)
edges_index = edges_index.transpose(dim0=0, dim1=1)

datas = []
data = Data(x=torch.from_numpy(node_[x]).float(), edge_index=edges_index, edge_attr=torch.from_numpy(edges_).float(),
            demand=torch.tensor(demand_[x]).unsqueeze(-1).float(),
            capcity=torch.tensor(capcity_[x]).unsqueeze(-1).float())
datas.append(data)

data_loder = DataLoader(datas, batch_size=1)


agent = Model(3, 128, 1, 16, conv_laysers=4).to(device)
agent.to(device)
folder = 'trained'
filepath = os.path.join(folder, '%s' % actor)

if os.path.exists(filepath):
    path1 = os.path.join(filepath, 'actor.pt')
    agent.load_state_dict(torch.load(path1, device))
if Greedy:
    batch = next(iter(data_loder))
    batch.to(device)
    agent.eval()
    #-------------------------------------------------------------------------------------------Greedy
    with torch.no_grad():
        tour, _ = agent(batch, n_nodes * 2,True)
        #cost = reward1(batch.x, tour.detach(), n_nodes)
        #print(cost)
        #print(tour)
#-------------------------------------------------------------------------------------------sampling1280
else:
    datas_ = []
    batch_size1 = 128  # sampling batch_size
    for y in range(1280):
        data = Data(x=torch.from_numpy(node_[x]).float(), edge_index=edges_index,
                    edge_attr=torch.from_numpy(edges_).float(),
                    demand=torch.tensor(demand_[x]).unsqueeze(-1).float(),
                    capcity=torch.tensor(capcity_[x]).unsqueeze(-1).float())
        datas_.append(data)
    dl = DataLoader(datas_, batch_size=batch_size1)

    min_tour=[]
    min_cost=100
    T=1.2#Temperature hyperparameters
    for batch in dl:
        with torch.no_grad():
            batch.to(device)
            tour1, _ = agent(batch, n_nodes * 2,False, T)
            cost = reward1(batch.x, tour1.detach(), n_nodes)
            id = np.array(cost.cpu()).argmin()
            m_cost=np.array(cost.cpu()).min()
            tour1=tour1.reshape(batch_size1,-1)
            if m_cost<min_cost:
                min_cost=m_cost
                min_tour=tour1[id]

    tour=min_tour.unsqueeze(-2)

print('Problem:VRP''%s' % n_nodes,'/ Average distance:', min_cost)
for i, (data, tour) in enumerate(zip(dl, tour)):
    routes = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)

print('_______________')

depot = data.x[0].cpu().numpy()
locs = data.x[1:].cpu().numpy()
demands = data.demand.cpu().numpy()
demands=demands[1:]
total = 0
for veh_number, r in enumerate(routes):
    route_demands = demands[r - 1]
    coords = locs[r - 1, :]
    xs, ys = coords.transpose()
    total_route_demand = sum(route_demands)
    total += total_route_demand*1000
    print(f'Маршрут {veh_number}, Груз на маршруте - {total_route_demand*1000}, Количество посещаемых узлов - {len(r)}')
print('Total demands''%s' % total)

# #--------------------------------------------------------------------------------------------
# for i, (data, tour) in enumerate(zip(data_loder, tour)):
#     if Greedy:
#         print(data.x,data.demand,tour)
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_vehicle_routes(data, tour, ax,Greedy, visualize_demands=False, demand_scale=20, round_demand=True)
#     else:
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_vehicle_routes(data, tour, ax,Greedy, visualize_demands=False, demand_scale=50, round_demand=True)

Problem:VRP40 / Average distance: 16.64761
[array([15, 11, 26, 38, 13, 31, 29, 16, 22, 30, 12], dtype=int64), array([ 3,  6,  8,  9, 17,  1,  7, 25], dtype=int64), array([37, 23,  5], dtype=int64), array([20, 10, 34, 14, 21, 32, 19,  4, 24], dtype=int64), array([35, 39, 28,  2], dtype=int64), array([18, 27, 33, 36], dtype=int64)]
_______________
Маршрут 0, Груз на маршруте - [7536.], Количество посещаемых узлов - 11
Маршрут 1, Груз на маршруте - [4536.0005], Количество посещаемых узлов - 8
Маршрут 2, Груз на маршруте - [1727.9999], Количество посещаемых узлов - 3
Маршрут 3, Груз на маршруте - [6734.], Количество посещаемых узлов - 9
Маршрут 4, Груз на маршруте - [2536.9998], Количество посещаемых узлов - 4
Маршрут 5, Груз на маршруте - [2635.0002], Количество посещаемых узлов - 4
Total demands[25706.]


### OR-Tools

In [189]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [233]:
node_ = np.loadtxt('VRP/test_data/vrp50_test_data.csv', dtype=float, delimiter=',')
node_ = node_[:n_nodes]

input_date = '2023-08-20' 
demand = get_orders_by_date(input_date, daily_orders, GNN = False)

C:\Users\skitl\AppData\Local\Temp\ipykernel_7296\3871925438.py:17: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  orders[0] = 0


In [242]:
print(demand_)

[[0.    0.458 0.462 0.508 0.705 0.27  0.677 0.5   0.545 0.637 0.658 0.703
  0.548 0.466 0.408 0.58  0.669 0.71  0.521 0.881 0.781 0.749 0.808 0.815
  0.963 1.001 0.836 0.54  0.847 1.035 0.635 0.917 0.865 0.62  0.724 0.663
  0.954 0.643 0.339 0.565]]


In [243]:
print(demand)

[0, 458, 462, 508, 705, 270, 677, 0, 545, 637, 658, 703, 548, 466, 408, 580, 669, 710, 521, 881, 781, 749, 808, 815, 963, 1001, 836, 540, 847, 1035, 635, 917, 865, 620, 724, 663, 954, 643, 339, 565]


In [244]:
import numpy as np
from scipy.spatial.distance import cdist

def euclidean_distances(points):
    # Преобразование массива точек в формат, понятный функции cdist
    points = np.squeeze(points)
    # Вычисление матрицы евклидовых расстояний с помощью функции cdist
    dist_matrix = cdist(points, points, metric='euclidean')
    multiplied_distances = (dist_matrix * 1000).astype(int)
    return multiplied_distances  # Преобразование в список для вывода

# Ваш входной массив точек
input_array = node_
# Вычисление матрицы евклидовых расстояний с помощью функции euclidean_distances
distances = euclidean_distances(input_array)

distances

array([[  0, 713, 672, ..., 275, 629, 785],
       [713,   0,  76, ..., 450, 925, 262],
       [672,  76,   0, ..., 422, 938, 332],
       ...,
       [275, 450, 422, ...,   0, 609, 512],
       [629, 925, 938, ..., 609,   0, 795],
       [785, 262, 332, ..., 512, 795,   0]])

In [245]:
edges_

array([[0.        ],
       [0.7138699 ],
       [0.67296804],
       ...,
       [0.51228862],
       [0.79582808],
       [0.        ]])

In [257]:
"""Capacited Vehicles Routing Problem (CVRP)."""

def create_data_model():
    """Stores the data for the problem."""

    data = {}
    data["distance_matrix"] = distances
    data["demands"] = demand
    data["vehicle_capacities"] = [8000, 8000, 8000, 8000]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data



def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance/1000}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )
  
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == "__main__":
    main()

Objective: 6746
Route for vehicle 0:
 0 Load(0) ->  3 Load(508) ->  4 Load(1213) ->  12 Load(1761) ->  38 Load(2100) ->  11 Load(2803) ->  13 Load(3269) ->  8 Load(3814) ->  9 Load(4451) ->  34 Load(5175) ->  26 Load(6011) ->  31 Load(6928) ->  15 Load(7508) ->  0 Load(7508)
Distance of the route: 2281m
Load of the route: 7508

Route for vehicle 1:
 0 Load(0) ->  18 Load(521) ->  32 Load(1386) ->  35 Load(2049) ->  30 Load(2684) ->  39 Load(3249) ->  17 Load(3959) ->  28 Load(4806) ->  1 Load(5264) ->  22 Load(6072) ->  2 Load(6534) ->  14 Load(6942) ->  7 Load(6942) ->  23 Load(7757) ->  0 Load(7757)
Distance of the route: 2189m
Load of the route: 7757

Route for vehicle 2:
 0 Load(0) ->  36 Load(954) ->  24 Load(1917) ->  20 Load(2698) ->  5 Load(2968) ->  25 Load(3969) ->  27 Load(4509) ->  21 Load(5258) ->  37 Load(5901) ->  10 Load(6559) ->  16 Load(7228) ->  0 Load(7228)
Distance of the route: 1302m
Load of the route: 7228

Route for vehicle 3:
 0 Load(0) ->  29 Load(1035) ->  33